# Code Reviewer Test

### Part two

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from scipy import stats as st

#### Read in the data

In [78]:
df=pd.read_csv('event_data.csv')
df.head()

,user_id,event_date,event_type,purchase_amount
0,c40e6a,2019-07-29 00:02:15,registration,NaN
1,a2b682,2019-07-29 00:04:46,registration,NaN
2,9ac888,2019-07-29 00:13:22,registration,NaN
3,93ff22,2019-07-29 00:16:47,registration,NaN
4,65ef85,2019-07-29 00:19:23,registration,NaN


In [81]:
df.loc[0:20,'user_id']

0     c40e6a
1     a2b682
2     9ac888
3     93ff22
4     65ef85
5     90852e
6     357151
7     71ac11
8     af679d
9     a48f29
10    b65930
11    956ad6
12    8aa5b4
13    5fb555
14    37fa41
15    b5787e
16    b2e16e
17    ca3c58
18    bea18b
19    48cac1
20    5290a3
Name: user_id, dtype: object

In [33]:
# convert event date from string to datetime object
df.event_date=pd.to_datetime(df.event_date)
df['week_of_event']=df.event_date.dt.isocalendar().week

In [34]:
df.head()

,user_id,event_date,event_type,purchase_amount,week_of_event
0,c40e6a,2019-07-29 00:02:15,registration,NaN,31
1,a2b682,2019-07-29 00:04:46,registration,NaN,31
2,9ac888,2019-07-29 00:13:22,registration,NaN,31
3,93ff22,2019-07-29 00:16:47,registration,NaN,31
4,65ef85,2019-07-29 00:19:23,registration,NaN,31


#### Cohorts based on week of registration

In [35]:
df[df.event_type=='registration'].head()

,user_id,event_date,event_type,purchase_amount,week_of_event
0,c40e6a,2019-07-29 00:02:15,registration,NaN,31
1,a2b682,2019-07-29 00:04:46,registration,NaN,31
2,9ac888,2019-07-29 00:13:22,registration,NaN,31
3,93ff22,2019-07-29 00:16:47,registration,NaN,31
4,65ef85,2019-07-29 00:19:23,registration,NaN,31


#### Number of unique users with cohort ID 33

In [36]:

df[(df['week_of_event']==33) & (df['event_type']=='registration')]['user_id'].nunique()

2045

#### Lifetime indicator

In [37]:
reg=pd.DataFrame(df[df['event_type']=='registration']. \
                 groupby(['user_id'])['week_of_event'].min()).rename(columns={'week_of_event':'registration'})

sim=pd.DataFrame(df[df['event_type']=='simple_event']. \
                 groupby(['user_id'])['week_of_event'].max()).rename(columns={'week_of_event':'simple_event'})

pur=pd.DataFrame(df[df['event_type']=='purchase']. \
                 groupby(['user_id'])['week_of_event'].max()).rename(columns={'week_of_event':'purchase'})

final_df=pd.concat([reg,sim,pur], axis=1)

summary_df=final_df.copy()

def get_simple_event_lifetime(row,x,y):
    return row[y]-row[x]

def get_purchase_lifetime(row,x,y):
    return row[y]-row[x]

summary_df['simple_event_lifetime']=summary_df.apply(get_simple_event_lifetime,axis=1,x='registration',y='simple_event')
summary_df['purchase_lifetime']=summary_df.apply(get_purchase_lifetime,axis=1,x='registration',y='purchase')
summary_df['registration_lifetime']=0

summary_df.head()

,registration,simple_event,purchase,simple_event_lifetime,purchase_lifetime,registration_lifetime
user_id,,,,,,
00049f,33,35,33,2,0,0
0005f9,35,35,<NA>,0,<NA>,0
00082c,31,34,33,3,2,0
000ff1,35,35,<NA>,0,<NA>,0
001d5d,34,35,35,1,1,0


#### Summary changes in Retention for cohorts depending on lifetime

In [38]:
summary_df.groupby('registration').agg({'registration':lambda x: (x.count()), \
                                       'simple_event':lambda x: (x.count()), \
                                       'purchase':lambda x: (x.count())})

,registration,simple_event,purchase
registration,,,
31,1975,1972,1075
32,1952,1946,1084
33,2045,2043,1035
34,1974,1969,752
35,2050,1549,337


#### Retention rate for cohorts for 3rd week

In [39]:
sum_df=summary_df[(summary_df['simple_event_lifetime']==3) | \
           (summary_df['purchase_lifetime']==3)]

In [40]:
agg_sum_df=sum_df.groupby('registration').agg({'registration':lambda x: (x.count()), \
                                       'simple_event':lambda x: (x.count()), \
                                       'purchase':lambda x: (x.count())})
agg_sum_df.head()

,registration,simple_event,purchase
registration,,,
31,499,499,368
32,705,705,517


#### 3 week retention rate for cohort with ID 32

In [41]:
round(((int(agg_sum_df.loc[32,'registration'])-int(agg_sum_df.loc[32,'purchase'])) \
/int(agg_sum_df.loc[32,'registration'])) *100,2)

26.67

#### Summary table for changes in Average Revenue Per Paying User

In [42]:

pd.DataFrame(df[(df['event_type']=='purchase')].groupby('week_of_event')['purchase_amount'].mean() \
/ df[(df['event_type']=='purchase')].groupby('week_of_event')['purchase_amount'].count()) \
.rename(columns={'purchase_amount':'ARPPU'})

,ARPPU
week_of_event,
31,0.089028
32,0.028202
33,0.021171
34,0.018306
35,0.017194


#### 3-week ARPPU of cohort with ID 31

In [43]:
cohorts = df[(df['event_type']=='registration')] #dataframe with only registration column
arppu_users = df[(df['event_type']=='purchase')] # dataframe with only purchase column
paying_registrants = pd.merge(cohorts, arppu_users, on="user_id") #merge both dataframes with common user_id

paying_registrants = paying_registrants[['user_id','event_type_x','week_of_event_x','event_type_y', \
                                                               'purchase_amount_y','week_of_event_y']] #obtain needed columns
paying_registrants['lifetime']=paying_registrants['week_of_event_y'] - paying_registrants['week_of_event_x'] #estimate lifetime
third_week_paying_registrants = paying_registrants[paying_registrants['lifetime']==3] #lifetime for 3 week
third_week_paying_registrants.head()

,user_id,event_type_x,week_of_event_x,event_type_y,purchase_amount_y,week_of_event_y,lifetime
1,c40e6a,registration,31,purchase,50.0,34,3
9,65ef85,registration,31,purchase,40.0,34,3
15,5fb555,registration,31,purchase,50.0,34,3
18,b5787e,registration,31,purchase,50.0,34,3
19,b2e16e,registration,31,purchase,20.0,34,3


In [44]:
round(((pd.DataFrame(third_week_paying_registrants.groupby('week_of_event_x') \
             ['purchase_amount_y'].mean()).loc[32,'purchase_amount_y'] /
pd.DataFrame(third_week_paying_registrants.groupby('week_of_event_x') \
             ['week_of_event_y'].count()).loc[32,'week_of_event_y'])*100),2)

13.93

In [67]:
paying_registrants.head()

,user_id,event_date_x,event_type_x,purchase_amount_x,week_of_event_x,event_date_y,event_type_y,purchase_amount_y,week_of_event_y,reg_to_purchase_time_in_secs
0,c40e6a,2019-07-29 00:02:15,registration,NaN,31,2019-08-10 11:40:06,purchase,10.0,32,1078671.0
1,c40e6a,2019-07-29 00:02:15,registration,NaN,31,2019-08-20 12:48:56,purchase,50.0,34,1946801.0
2,a2b682,2019-07-29 00:04:46,registration,NaN,31,2019-08-06 23:18:39,purchase,30.0,32,774833.0
3,9ac888,2019-07-29 00:13:22,registration,NaN,31,2019-08-02 02:07:01,purchase,50.0,31,352419.0
4,9ac888,2019-07-29 00:13:22,registration,NaN,31,2019-08-04 02:14:50,purchase,50.0,31,525688.0


#### Median Time between user and first purchase

In [70]:
cohorts = df[(df['event_type']=='registration')] #dataframe with only registration column
arppu_users = df[(df['event_type']=='purchase')] # dataframe with only purchase column
paying_registrants = pd.merge(cohorts, arppu_users, on="user_id") #merge both dataframes with common user_id
paying_registrants['lifetime']=paying_registrants['week_of_event_y'] - paying_registrants['week_of_event_x']
paying_registrants['reg_to_purchase_time_in_secs'] = (paying_registrants['event_date_y']-paying_registrants \
                                                      ['event_date_x']).dt.total_seconds() #extract time_difference_in_sec
reg_to_first_purchase=paying_registrants[paying_registrants['lifetime']==1] #first purchase

In [75]:
reg_to_first_purchase['reg_to_purchase_time_in_secs'].median()

574234.5